In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
pip install mediapipe opencv-python tqdm numpy


In [ ]:
# --- Batch Keypoint Extraction (set START_IDX / END_IDX below) ---

import os
import cv2
import numpy as np
from tqdm import tqdm
import mediapipe as mp

# ====== CONFIG ======
DATASET_ROOT = "/content/drive/MyDrive/AFML/Dataset/WLASL_Videos"
START_IDX = 301   # <-- change per notebook: 200, 300, 400
END_IDX   = 400   # <-- change per notebook: 300, 400, 500
# ====================

# OpenCV can oversubscribe threads on Colab; keep it tame
try:
    cv2.setNumThreads(1)
except Exception:
    pass

def list_gloss_dirs(root):
    items = sorted(os.listdir(root))
    dirs = [d for d in items if os.path.isdir(os.path.join(root, d))]
    return dirs

def extract_keypoints_from_frame(results_pose, results_hands):
    keypoints = []

    # Pose: 33 landmarks (x,y). Use 0s if missing.
    if results_pose and results_pose.pose_landmarks:
        for lm in results_pose.pose_landmarks.landmark:
            keypoints.extend([lm.x, lm.y])
    else:
        keypoints.extend([0.0, 0.0] * 33)

    # Hands: map to LEFT / RIGHT using handedness labels to keep them stable
    left_hand  = [0.0, 0.0] * 21
    right_hand = [0.0, 0.0] * 21

    if results_hands and results_hands.multi_hand_landmarks:
        # Build (label, landmarks) pairs if handedness info exists
        labeled = []
        if results_hands.multi_handedness and \
           len(results_hands.multi_handedness) == len(results_hands.multi_hand_landmarks):
            for hmeta, hland in zip(results_hands.multi_handedness, results_hands.multi_hand_landmarks):
                label = hmeta.classification[0].label  # 'Left' or 'Right'
                labeled.append((label, hland))
        else:
            # Fallback to whatever order MediaPipe gives
            for hland in results_hands.multi_hand_landmarks:
                labeled.append(("Unknown", hland))

        for label, hand_land in labeled:
            pts = []
            for lm in hand_land.landmark:
                pts.extend([lm.x, lm.y])
            if label == "Left":
                left_hand = pts
            elif label == "Right":
                right_hand = pts
            else:
                # If unknown and a slot is empty, fill right then left
                if right_hand == [0.0, 0.0] * 21:
                    right_hand = pts
                else:
                    left_hand = pts

    return keypoints + left_hand + right_hand

def process_video(video_path, pose_model, hands_model):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # BGR -> RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pose_res = pose_model.process(frame_rgb)
        hand_res = hands_model.process(frame_rgb)
        frames.append(extract_keypoints_from_frame(pose_res, hand_res))
    cap.release()
    return np.array(frames, dtype=np.float32)

def main():
    all_glosses = list_gloss_dirs(DATASET_ROOT)
    total = len(all_glosses)
    start = max(0, START_IDX)
    end = min(END_IDX, total)

    if start >= end:
        raise ValueError(f"Empty range: START_IDX={START_IDX}, END_IDX={END_IDX}, total glosses={total}")

    target_glosses = all_glosses[start:end]
    print(f"Total gloss folders: {total}")
    print(f"Processing index range [{start}:{end}) → {len(target_glosses)} glosses")

    # Create MediaPipe models once per notebook
    with mp.solutions.pose.Pose() as pose, mp.solutions.hands.Hands(max_num_hands=2) as hands:
        for gloss in tqdm(target_glosses, desc=f"Processing glosses {start}-{end-1}"):
            gloss_path = os.path.join(DATASET_ROOT, gloss)
            print(f"\n➡ Gloss: {gloss}")

            try:
                files = os.listdir(gloss_path)
            except Exception as e:
                print(f"⚠️  Skipping {gloss} (cannot list dir): {e}")
                continue

            mp4s = [f for f in files if f.lower().endswith(".mp4")]
            for file in mp4s:
                video_path = os.path.join(gloss_path, file)
                output_path = os.path.splitext(video_path)[0] + ".npy"

                if os.path.exists(output_path):
                    print(f"✅ Exists: {file}")
                    continue

                print(f"🎥 Extracting: {file}")
                try:
                    frames_np = process_video(video_path, pose, hands)
                    # Ensure parent dir exists (should already)
                    np.save(output_path, frames_np)
                    print(f"💾 Saved → {output_path}")
                except Exception as e:
                    print(f"❌ Failed on {file}: {e}")

    print("\n✅ DONE — Batch keypoints extracted")

if __name__ == "__main__":
    main()


Total gloss folders: 500
Processing index range [301:400) → 99 glosses


Processing glosses 301-399:   0%|          | 0/99 [00:00<?, ?it/s]


➡ Gloss: music


Processing glosses 301-399:   1%|          | 1/99 [00:00<00:25,  3.77it/s]

✅ Exists: 9815.mp4
✅ Exists: MUSIC-1584.mp4
✅ Exists: 48609.mp4
✅ Exists: music.mp4
✅ Exists: SignSchool Music, Sing-N2r3NCId7AE.mp4
✅ Exists: 1468717488.5224.mp4
✅ Exists: 7756.mp4

➡ Gloss: name


...

💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/sit/7822.npy

➡ Gloss: sleep
🎥 Extracting: sleep.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/sleep/sleep.npy
🎥 Extracting: SignSchool Sleep.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/sleep/SignSchool Sleep.npy
🎥 Extracting: 1468767820.767.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/sleep/1468767820.767.npy
🎥 Extracting: 23907.mp4


Processing glosses 301-399:  99%|█████████▉| 98/99 [1:24:05<01:40, 100.87s/it]

💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/sleep/23907.npy

➡ Gloss: slow
🎥 Extracting: SLOW-261.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/slow/SLOW-261.npy
🎥 Extracting: 486598.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/slow/486598.npy
🎥 Extracting: slow.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/slow/slow.npy
🎥 Extracting: SignSchool Slow-fUy_ttG5QxY.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/slow/SignSchool Slow-fUy_ttG5QxY.npy
🎥 Extracting: 1468767957.1850.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/slow/1468767957.1850.npy
🎥 Extracting: 1522767290.120.mp4
💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/slow/1522767290.120.npy
🎥 Extracting: 14352.mp4


Processing glosses 301-399: 100%|██████████| 99/99 [1:24:51<00:00, 51.43s/it]

💾 Saved → /content/drive/MyDrive/AFML/Dataset/WLASL_Videos/slow/14352.npy

✅ DONE — Batch keypoints extracted


In [ ]:
!pip install torch torchvision torchaudio


In [ ]:
# ============================================================================
# Stronger CNN + Mamba model + Training Loop for WLASL (498 classes)
# Assumes:
#   - train_loader and val_loader are already defined
#   - Each batch: inputs (B, T, 150), labels (B,)
#   - Using Google Drive path for saving best model
# ============================================================================

import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ----------------------------------------------------------------------------
# Mamba Block (simplified, pure PyTorch)
# ----------------------------------------------------------------------------
class MambaBlock(nn.Module):
    """
    Simplified Mamba-style block for temporal modeling.
    Works on sequences of shape (B, T, D).
    """
    def __init__(self, d_model: int, d_state: int = 32, expand: int = 2, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_inner = d_model * expand

        # Input projection -> split into (u, gate)
        self.in_proj = nn.Linear(d_model, self.d_inner * 2)

        # State convolution over time (depthwise)
        self.conv = nn.Conv1d(
            self.d_inner, self.d_inner,
            kernel_size=3, padding=1, groups=self.d_inner
        )

        self.out_proj = nn.Linear(self.d_inner, d_model)
        self.act = nn.SiLU()
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        """
        x: (B, T, D)
        """
        residual = x
        x = self.norm(x)

        # Input projection + gating
        u, gate = self.in_proj(x).chunk(2, dim=-1)  # (B, T, d_inner) each
        gate = torch.sigmoid(gate)

        # Depthwise conv over time
        u = u.transpose(1, 2)       # (B, d_inner, T)
        u = self.conv(u)            # (B, d_inner, T)
        u = u.transpose(1, 2)       # (B, T, d_inner)

        u = self.act(u) * gate
        u = self.out_proj(u)        # (B, T, D)
        u = self.dropout(u)

        return residual + u


# ----------------------------------------------------------------------------
# Attention Pooling over time
# ----------------------------------------------------------------------------
class AttentionPool(nn.Module):
    def __init__(self, d_model: int):
        super().__init__()
        self.query = nn.Parameter(torch.randn(d_model))

    def forward(self, x):
        """
        x: (B, T, D)
        returns: (B, D)
        """
        scores = torch.einsum('btd,d->bt', x, self.query) / math.sqrt(x.size(-1))
        weights = F.softmax(scores, dim=1)      # (B, T)
        pooled = torch.einsum('btd,bt->bd', x, weights)
        return pooled


# ----------------------------------------------------------------------------
# Stronger CNN + Mamba model
# ----------------------------------------------------------------------------
class CNNMambaSignModel(nn.Module):
    def __init__(
        self,
        input_dim: int = 150,
        num_classes: int = 498,
        d_model: int = 256,
        num_cnn_layers: int = 2,
        num_mamba_layers: int = 2,
        dropout: float = 0.2,
    ):
        super().__init__()
        self.d_model = d_model

        # Project 150-dim keypoints to d_model
        self.input_proj = nn.Linear(input_dim, d_model)

        # Temporal CNN stack (1D conv over frames)
        cnn_layers = []
        for _ in range(num_cnn_layers):
            cnn_layers.append(
                nn.Sequential(
                    nn.Conv1d(d_model, d_model, kernel_size=3, padding=1),
                    nn.BatchNorm1d(d_model),
                    nn.SiLU(),
                    nn.Dropout(dropout),
                )
            )
        self.cnn_layers = nn.ModuleList(cnn_layers)

        # Mamba stack for deeper temporal modeling
        mamba_layers = []
        for _ in range(num_mamba_layers):
            mamba_layers.append(
                MambaBlock(d_model=d_model, d_state=32, expand=2, dropout=dropout)
            )
        self.mamba_layers = nn.ModuleList(mamba_layers)

        # Temporal attention pooling
        self.attn_pool = AttentionPool(d_model)

        # Final classifier head
        self.fc = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, num_classes),
        )

    def forward(self, x):
        """
        x: (B, T, 150)
        """
        # Project to d_model
        x = self.input_proj(x)          # (B, T, D)

        # CNN expects (B, D, T)
        x = x.transpose(1, 2)           # (B, D, T)
        for layer in self.cnn_layers:
            residual = x
            out = layer(x)
            x = out + residual          # residual CNN

        # Back to (B, T, D)
        x = x.transpose(1, 2)           # (B, T, D)

        # Mamba layers
        for layer in self.mamba_layers:
            x = layer(x)                # (B, T, D)

        # Attention pooling over time
        x = self.attn_pool(x)           # (B, D)

        # Classifier
        logits = self.fc(x)             # (B, num_classes)
        return logits


# ----------------------------------------------------------------------------
# Accuracy helpers (Top-1, Top-5)
# ----------------------------------------------------------------------------
def accuracy_topk(logits, targets, topk=(1, 5)):
    """
    logits: (B, C)
    targets: (B,)
    Returns: list of accuracies for each k in topk
    """
    maxk = max(topk)
    batch_size = targets.size(0)

    _, pred = logits.topk(maxk, dim=1, largest=True, sorted=True)  # (B, maxk)
    pred = pred.t()                                                # (maxk, B)
    correct = pred.eq(targets.view(1, -1).expand_as(pred))        # (maxk, B)

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append((correct_k * (100.0 / batch_size)).item())
    return res  # [top1, top5, ...]


# ----------------------------------------------------------------------------
# Instantiate model, loss, optimizer, scheduler
# ----------------------------------------------------------------------------
num_classes = 498
input_dim = 150
num_epochs = 60

model = CNNMambaSignModel(
    input_dim=input_dim,
    num_classes=num_classes,
    d_model=256,
    num_cnn_layers=2,
    num_mamba_layers=2,
    dropout=0.2,
).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    weight_decay=1e-4,
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=num_epochs
)

print(model)


# ----------------------------------------------------------------------------
# Training & Validation Loop
# ----------------------------------------------------------------------------
best_val_top1 = 0.0
save_path = "/content/drive/MyDrive/AFML/Dataset/cnn_mamba_wlasl_highacc.pt"

for epoch in range(1, num_epochs + 1):
    # ------------------------
    # Train
    # ------------------------
    model.train()
    running_loss = 0.0
    running_top1 = 0.0
    running_top5 = 0.0
    total_train = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)      # (B, T, 150)
        labels = labels.to(device)      # (B,)

        optimizer.zero_grad()

        logits = model(inputs)          # (B, num_classes)
        loss = criterion(logits, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        batch_size = labels.size(0)
        total_train += batch_size
        running_loss += loss.item() * batch_size

        top1, top5 = accuracy_topk(logits, labels, topk=(1, 5))
        running_top1 += top1 * batch_size / 100.0
        running_top5 += top5 * batch_size / 100.0

    train_loss = running_loss / total_train
    train_top1 = (running_top1 / total_train) * 100.0
    train_top5 = (running_top5 / total_train) * 100.0

    # ------------------------
    # Validation
    # ------------------------
    model.eval()
    val_loss_sum = 0.0
    val_top1_sum = 0.0
    val_top5_sum = 0.0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            logits = model(inputs)
            loss = criterion(logits, labels)

            batch_size = labels.size(0)
            total_val += batch_size
            val_loss_sum += loss.item() * batch_size

            top1, top5 = accuracy_topk(logits, labels, topk=(1, 5))
            val_top1_sum += top1 * batch_size / 100.0
            val_top5_sum += top5 * batch_size / 100.0

    val_loss = val_loss_sum / total_val
    val_top1 = (val_top1_sum / total_val) * 100.0
    val_top5 = (val_top5_sum / total_val) * 100.0

    # Step LR scheduler
    scheduler.step()

    print(f"\nEpoch {epoch}/{num_epochs}")
    print(f"Train loss: {train_loss:.4f} | Top1: {train_top1:.2f}% | Top5: {train_top5:.2f}%")
    print(f"Val   loss: {val_loss:.4f} | Top1: {val_top1:.2f}% | Top5: {val_top5:.2f}%")

    # Save best model by val_top1
    if val_top1 > best_val_top1:
        best_val_top1 = val_top1
        torch.save(model.state_dict(), save_path)
        print(f"✅ New best model saved with val_top1={best_val_top1:.2f}% → {save_path}")
    else:
        print(f"No improvement. Best val_top1 so far: {best_val_top1:.2f}%")

print("\nTraining finished.")
print(f"Best validation Top-1 accuracy: {best_val_top1:.2f}%")


Using device: cuda
CNNMambaSignModel(
  (input_proj): Linear(in_features=150, out_features=256, bias=True)
  (cnn_layers): ModuleList(
    (0-1): 2 x Sequential(
      (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (mamba_layers): ModuleList(
    (0-1): 2 x MambaBlock(
      (in_proj): Linear(in_features=256, out_features=1024, bias=True)
      (conv): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
      (out_proj): Linear(in_features=512, out_features=256, bias=True)
      (act): SiLU()
      (dropout): Dropout(p=0.2, inplace=False)
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (attn_pool): AttentionPool()
  (fc): Sequential(
    (0): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_

In [ ]:
import os
import math
import numpy as np
from typing import List, Tuple

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import Counter

# -----------------------------
# CONFIG
# -----------------------------
DATASET_ROOT = "/content/drive/MyDrive/AFML/Dataset/WLASL_Videos"

# If your .npy files are inside a "preprocessed" subfolder, change this:
# PREPROCESSED_ROOT = os.path.join(DATASET_ROOT, "preprocessed")
PREPROCESSED_ROOT = DATASET_ROOT   # adjust if needed

BATCH_SIZE = 32
NUM_EPOCHS = 60
SEQ_LEN = 64          # <<< fixed number of frames per sample
SAVE_PATH = "/content/drive/MyDrive/AFML/Dataset/cnn_mamba_wlasl_highacc.pt"

# New hyperparameters
D_MODEL           = 192
NUM_CNN_LAYERS    = 2
NUM_MAMBA_LAYERS  = 2
DROPOUT_RATE      = 0.3
LR                = 5e-4
WEIGHT_DECAY      = 5e-4
LABEL_SMOOTHING   = 0.1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================================
# 1. DATASET + DATALOADER (with pad/crop to SEQ_LEN)
# ============================================================================

class WLASLKeypointDataset(Dataset):
    def __init__(self, root: str, samples: List[Tuple[str, int]], seq_len: int = 64):
        """
        root: base folder where .npy files exist
        samples: list of (relative_path, label)
        seq_len: fixed number of frames (T) after pad/crop
        """
        self.root = root
        self.samples = samples
        self.seq_len = seq_len

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        rel_path, label = self.samples[idx]
        full_path = os.path.join(self.root, rel_path)

        keypoints = np.load(full_path)   # original shape: (T, 150)
        T, D = keypoints.shape

        # --- Pad or crop to self.seq_len ---
        if T > self.seq_len:
            # center crop
            start = (T - self.seq_len) // 2
            keypoints = keypoints[start:start + self.seq_len]
        elif T < self.seq_len:
            pad_len = self.seq_len - T
            pad = np.zeros((pad_len, D), dtype=keypoints.dtype)
            keypoints = np.concatenate([keypoints, pad], axis=0)
        # now shape = (seq_len, 150)

        keypoints = torch.tensor(keypoints, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        return keypoints, label


# Build file list + label map
glosses = sorted([
    d for d in os.listdir(PREPROCESSED_ROOT)
    if os.path.isdir(os.path.join(PREPROCESSED_ROOT, d))
])

label_map = {gloss: i for i, gloss in enumerate(glosses)}

all_files = []
missing_count = 0

print("Scanning gloss folders and checking .npy files...")
for gloss in tqdm(glosses):
    gloss_folder = os.path.join(PREPROCESSED_ROOT, gloss)
    for f in os.listdir(gloss_folder):
        if not f.endswith(".npy"):
            continue
        rel_path = f"{gloss}/{f}"
        full_path = os.path.join(PREPROCESSED_ROOT, rel_path)
        if os.path.isfile(full_path):
            label = label_map[gloss]
            all_files.append((rel_path, label))
        else:
            missing_count += 1

print(f"\nNum valid samples total: {len(all_files)}")
print(f"Num missing/invalid files skipped: {missing_count}")

# Filter out classes with only one sample before splitting
labels_for_split = [lbl for _, lbl in all_files]
label_counts = Counter(labels_for_split)
single_sample_labels = {label for label, count in label_counts.items() if count < 2}

filtered_all_files = [
    (rel_path, label) for rel_path, label in all_files
    if label not in single_sample_labels
]

# Identify which of the original glosses are still present
unique_original_labels_in_filtered = sorted(list(set([label for _, label in filtered_all_files])))
present_gloss_names = [glosses[lbl] for lbl in unique_original_labels_in_filtered]
new_label_map = {gloss_name: i for i, gloss_name in enumerate(present_gloss_names)}

# Update labels in filtered_all_files to use the new contiguous labels
final_all_files = [
    (rel_path, new_label_map[glosses[old_label]]) for rel_path, old_label in filtered_all_files
]

num_classes = len(present_gloss_names)
print(f"Num classes (after filtering single-sample): {num_classes}")
print(f"Num samples after filtering: {len(final_all_files)}")

# Train/val split (stratified) with filtered data
labels_for_split = [lbl for _, lbl in final_all_files]
train_samples, val_samples = train_test_split(
    final_all_files,
    test_size=0.15,
    random_state=42,
    stratify=labels_for_split
)

print(f"Num train samples: {len(train_samples)}")
print(f"Num val samples  : {len(val_samples)}")

train_dataset = WLASLKeypointDataset(PREPROCESSED_ROOT, train_samples, seq_len=SEQ_LEN)
val_dataset   = WLASLKeypointDataset(PREPROCESSED_ROOT, val_samples,   seq_len=SEQ_LEN)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True,
    drop_last=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)

# Quick sanity check
batch = next(iter(train_loader))
print("Batch keypoints shape:", batch[0].shape)  # (B, 64, 150)
print("Batch labels shape   :", batch[1].shape)


# ============================================================================
# 2. MODEL: CNN + Mamba-style temporal block
# ============================================================================

class MambaBlock(nn.Module):
    """
    Simplified Mamba-style block for temporal modeling.
    Input:  (B, T, D)
    Output: (B, T, D)
    """
    def __init__(self, d_model: int, d_state: int = 32, expand: int = 2, dropout: float = 0.1):
        super().__init__()
        self.d_model = d_model
        self.d_state = d_state
        self.d_inner = d_model * expand * 2  # 2x for (u, gate)

        # Input projection → (u, gate)
        self.in_proj = nn.Linear(d_model, self.d_inner)

        # Depthwise temporal conv on u
        self.conv = nn.Conv1d(
            self.d_inner // 2,
            self.d_inner // 2,
            kernel_size=3,
            padding=1,
            groups=self.d_inner // 2,
        )

        self.out_proj = nn.Linear(self.d_inner // 2, d_model)
        self.act = nn.SiLU()
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        """
        x: (B, T, D)
        """
        residual = x
        x = self.norm(x)

        h = self.in_proj(x)                   # (B, T, d_inner)
        u, gate = h.chunk(2, dim=-1)          # (B, T, d_inner/2) each
        gate = torch.sigmoid(gate)

        # depthwise conv over time
        u = u.transpose(1, 2)                 # (B, d_inner/2, T)
        u = self.conv(u)                      # (B, d_inner/2, T)
        u = u.transpose(1, 2)                 # (B, T, d_inner/2)

        u = self.act(u) * gate
        u = self.out_proj(u)                  # (B, T, D)
        u = self.dropout(u)

        return residual + u


class AttentionPool(nn.Module):
    def __init__(self, d_model: int):
        super().__init__()
        self.query = nn.Parameter(torch.randn(d_model))

    def forward(self, x):
        """
        x: (B, T, D)
        returns: (B, D)
        """
        scores = torch.einsum("btd,d->bt", x, self.query) / math.sqrt(x.size(-1))
        weights = F.softmax(scores, dim=1)          # (B, T)
        pooled = torch.einsum("btd,bt->bd", x, weights)
        return pooled


class CNNMambaSignModel(nn.Module):
    def __init__(
        self,
        input_dim: int = 150,
        num_classes: int = 498,
        d_model: int = 256,
        num_cnn_layers: int = 2,
        num_mamba_layers: int = 2,
        dropout: float = 0.2,
    ):
        super().__init__()
        self.d_model = d_model

        # Project raw keypoints (150) to model dim (256)
        self.input_proj = nn.Linear(input_dim, d_model)

        # Temporal CNN stack: Conv1d over time
        cnn_layers = []
        for _ in range(num_cnn_layers):
            cnn_layers.append(
                nn.Sequential(
                    nn.Conv1d(d_model, d_model, kernel_size=3, padding=1),
                    nn.BatchNorm1d(d_model),
                    nn.SiLU(),
                    nn.Dropout(dropout),
                )
            )
        self.cnn_layers = nn.ModuleList(cnn_layers)

        # Mamba-style temporal stack
        mamba_layers = []
        for _ in range(num_mamba_layers):
            mamba_layers.append(
                MambaBlock(d_model=d_model, d_state=32, expand=2, dropout=dropout)
            )
        self.mamba_layers = nn.ModuleList(mamba_layers)

        # Attention pooling over time
        self.attn_pool = AttentionPool(d_model)

        # Classifier head
        self.fc = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Dropout(dropout),
            nn.Linear(d_model, d_model),
            nn.SiLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, num_classes),
        )

    def forward(self, x):
        """
        x: (B, T, 150)
        """
        x = self.input_proj(x)          # (B, T, D)

        # CNN expects (B, D, T)
        x = x.transpose(1, 2)           # (B, D, T)
        for layer in self.cnn_layers:
            residual = x
            out = layer(x)
            x = out + residual          # residual connection

        # Back to (B, T, D)
        x = x.transpose(1, 2)           # (B, T, D)

        # Mamba layers
        for layer in self.mamba_layers:
            x = layer(x)                # (B, T, D)

        # Attention pool over time
        x = self.attn_pool(x)           # (B, D)

        # Classifier
        logits = self.fc(x)             # (B, num_classes)
        return logits


# ============================================================================
# 3. TRAINING SETUP
# ============================================================================

def accuracy_topk(logits, targets, topk=(1, 5)):
    """
    logits: (B, C)
    targets: (B,)
    returns: [top1, top5, ...] in %
    """
    maxk = max(topk)
    batch_size = targets.size(0)

    _, pred = logits.topk(maxk, dim=1, largest=True, sorted=True)  # (B, maxk)
    pred = pred.t()                                                # (maxk, B)
    correct = pred.eq(targets.view(1, -1).expand_as(pred))        # (maxk, B)

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append((correct_k * (100.0 / batch_size)).item()))
    return res


model = CNNMambaSignModel(
    input_dim=150,
    num_classes=num_classes,
    d_model=D_MODEL,
    num_cnn_layers=NUM_CNN_LAYERS,
    num_mamba_layers=NUM_MAMBA_LAYERS,
    dropout=DROPOUT_RATE,
).to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=LABEL_SMOOTHING)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LR,
    weight_decay=WEIGHT_DECAY,
)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, T_max=NUM_EPOCHS
)

print(model)

# ============================================================================
# 4. TRAINING LOOP
# ============================================================================

best_val_top1 = 0.0

for epoch in range(1, NUM_EPOCHS + 1):
    # ---------------- Train ----------------
    model.train()
    running_loss = 0.0
    running_top1 = 0.0
    running_top5 = 0.0
    total_train = 0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)      # (B, 64, 150)
        labels = labels.to(device)      # (B,)

        optimizer.zero_grad()
        logits = model(inputs)
        loss = criterion(logits, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        batch_size = labels.size(0)
        total_train += batch_size
        running_loss += loss.item() * batch_size

        top1, top5 = accuracy_topk(logits, labels, topk=(1, 5))
        running_top1 += top1 * batch_size / 100.0
        running_top5 += top5 * batch_size / 100.0

    train_loss = running_loss / total_train
    train_top1 = (running_top1 / total_train) * 100.0
    train_top5 = (running_top5 / total_train) * 100.0

    # ---------------- Val ----------------
    model.eval()
    val_loss_sum = 0.0
    val_top1_sum = 0.0
    val_top5_sum = 0.0
    total_val = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            logits = model(inputs)
            loss = criterion(logits, labels)

            batch_size = labels.size(0)
            total_val += batch_size
            val_loss_sum += loss.item() * batch_size

            top1, top5 = accuracy_topk(logits, labels, topk=(1, 5))
            val_top1_sum += top1 * batch_size / 100.0
            val_top5_sum += top5 * batch_size / 100.0

    val_loss = val_loss_sum / total_val
    val_top1 = (val_top1_sum / total_val) * 100.0
    val_top5 = (val_top5_sum / total_val) * 100.0

    scheduler.step()

    print(f"\nEpoch {epoch}/{NUM_EPOCHS}")
    print(f"Train loss: {train_loss:.4f} | Top1: {train_top1:.2f}% | Top5: {train_top5:.2f}%")
    print(f"Val   loss: {val_loss:.4f} | Top1: {val_top1:.2f}% | Top5: {val_top5:.2f}%")

    # Save best model
    if val_top1 > best_val_top1:
        best_val_top1 = val_top1
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"\u2705 New best model saved with val_top1={best_val_top1:.2f}% \u2192 {SAVE_PATH}")
    else:
        print(f"No improvement. Best val_top1 so far: {best_val_top1:.2f}%")

print("\nTraining finished.")
print(f"Best validation Top-1 accuracy: {best_val_top1:.2f}%")


Using device: cuda
Scanning gloss folders and checking .npy files...


100%|██████████| 500/500 [00:01<00:00, 433.77it/s]


Num valid samples total: 2065
Num missing/invalid files skipped: 0
Num classes (after filtering single-sample): 256
Num samples after filtering: 2064
Num train samples: 1754
Num val samples  : 310


Batch keypoints shape: torch.Size([32, 64, 150])
Batch labels shape   : torch.Size([32])
CNNMambaSignModel(
  (input_proj): Linear(in_features=150, out_features=256, bias=True)
  (cnn_layers): ModuleList(
    (0-1): 2 x Sequential(
      (0): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU()
      (3): Dropout(p=0.2, inplace=False)
    )
  )
  (mamba_layers): ModuleList(
    (0-1): 2 x MambaBlock(
      (in_proj): Linear(in_features=256, out_features=1024, bias=True)
      (conv): Conv1d(512, 512, kernel_size=(3,), stride=(1,), padding=(1,), groups=512)
      (out_proj): Linear(in_features=512, out_features=256, bias=True)
      (act): SiLU()
      (dropout): Dropout(p=0.2, inplace=False)
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
  )
  (attn_pool): AttentionPool()
  (fc): Sequential(
    (0): LayerNorm((256,), eps=1e-05, elementwise_a